In [1]:
import pandas as pd
import emoji

In [109]:
df=pd.read_csv("CSDS351_Project_Data.csv", encoding= 'unicode_escape')
data=pd.concat([df['text'],df['sentiment']],axis=1)
data

,text,sentiment
0,Animals defecating fornicating and playing are...,Positive
1,âI regret nothing! Do hear me? Nothing!â,Positive
2,It just wanted to be a part of them,Negative
3,feed me or i press enter,Negative
4,They call him Gordie.,Neutral
...,...,...
620,Are groups like ASMP PPA etc. relevant anymore?,Neutral
621,What are your thoughts on the artistic ethics ...,Neutral
622,Album Share Tuesday: April 04 2023,Neutral
623,Official Question Thread! Ask /r/photography a...,Neutral


In [111]:
new_data=pd.read_csv('test_data2.csv', encoding= 'unicode_escape')
data= data.append(new_data, ignore_index=True)
data

,text,sentiment
0,Animals defecating fornicating and playing are...,Positive
1,âI regret nothing! Do hear me? Nothing!â,Positive
2,It just wanted to be a part of them,Negative
3,feed me or i press enter,Negative
4,They call him Gordie.,Neutral
...,...,...
1245,Cancer and heart disease vaccines ready by en...,Positive
1246,The miracle that disrupts order: mathematici...,Positive
1247,Avoid using free public phone charging station...,Negative
1248,"Washington Apple Store Robbed of $500,000 in i...",Negative


In [112]:
data = data.apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
data = data.apply(lambda x: x.apply(lambda y: emoji.demojize(y)))

In [113]:
data

,text,sentiment
0,Animals defecating fornicating and playing are...,Positive
1,I regret nothing! Do hear me? Nothing!,Positive
2,It just wanted to be a part of them,Negative
3,feed me or i press enter,Negative
4,They call him Gordie.,Neutral
...,...,...
1245,Cancer and heart disease vaccines ready by end...,Positive
1246,The miracle that disrupts order: mathematician...,Positive
1247,Avoid using free public phone charging station...,Negative
1248,"Washington Apple Store Robbed of $500,000 in i...",Negative


In [41]:
from transformers import pipeline

In [48]:
def label_to_pred(label: str):
    """Label should be one of LABEL_0, LABEL_1 or LABEL_2
    """
    idx = label.split("_")[1]
    if not idx.isdigit():
        return f"Bad prediction label: {label}"

    label_idx = int(idx)
    mappings = ["Negative", "Neutral", "Positive"]
    return mappings[label_idx]

In [6]:
from transformers import pipeline, AutoTokenizer
from bs4 import BeautifulSoup
import contractions
import re
import unicodedata
import string
import nltk
from nltk.tokenize import ToktokTokenizer
nltk.download('stopwords')
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('not')

c:\Users\vshar\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vshar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def remove_html_tags(text):
    return BeautifulSoup(text, 'html.parser').get_text()

def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

def expand_contractions(text):
    expanded_words = []   
    for word in text.split():
    # using contractions.fix to expand the shortened words
        expanded_words.append(contractions.fix(word))  
    new_text = ' '.join(expanded_words)
    return new_text

def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

def remove_numbers(text):
    # define the pattern to keep
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', text)

def remove_punctuation(text):
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

def get_stem(text):
    stemmer = nltk.porter.PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

def remove_stopwords(text):
    # convert sentence into token of words
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    # check in lowercase 
    t = [token for token in tokens if token.lower() not in stopword_list]
    text = ' '.join(t)    
    return text

def remove_extra_whitespace_tabs(text):
    #pattern = r'^\s+$|\s+$'
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()

def to_lowercase(text):
    return text.lower()


In [8]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer 

def get_lem(text):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(text, pos='n')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vshar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vshar\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [46]:
data.head()

,text,sentiment
0,animals defecating fornicating playing engagin...,Positive
1,âI regret nothing! Do hear me? Nothing!â,Positive
2,It just wanted to be a part of them,Negative
3,feed me or i press enter,Negative
4,They call him Gordie.,Neutral


In [114]:
for i in range(1250):
    input=remove_html_tags(data['text'][i])
    input=remove_accented_chars(input)
    input=expand_contractions(input)
    input=remove_special_characters(input)
    #input=remove_numbers(input)
    #input=remove_punctuation(input)
    input=get_lem(input)
    #input=remove_stopwords(input)
    input=remove_extra_whitespace_tabs(input)
    input=to_lowercase(input)
    data['text'][i]=input

C:\Users\vshar\AppData\Local\Temp\ipykernel_28160\2268567960.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, 'html.parser').get_text()


In [115]:
data

,text,sentiment
0,animals defecating fornicating and playing are...,Positive
1,i regret nothing! do hear me? nothing!,Positive
2,it just wanted to be a part of them,Negative
3,feed me or i press enter,Negative
4,they call him gordie.,Neutral
...,...,...
1245,cancer and heart disease vaccines ready by end...,Positive
1246,the miracle that disrupts order: mathematician...,Positive
1247,avoid using free public phone charging station...,Negative
1248,"washington apple store robbed of 500,000 in ip...",Negative


In [116]:
accuracy=0
count=0
predicted_labels=[]
classifier = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment")
for i in range(1250):
    res=classifier(data["text"][i])[0]
    label=label_to_pred(res["label"])
    predicted_labels.append(label)
    if (label==data["sentiment"][i]):
        count=count+1
accuracy=count/1250


In [117]:
accuracy_scores=[]
accuracy_scores.append(accuracy)

In [118]:
data['Model1']=predicted_labels
data.head()

,text,sentiment,Model1
0,animals defecating fornicating and playing are...,Positive,Negative
1,i regret nothing! do hear me? nothing!,Positive,Positive
2,it just wanted to be a part of them,Negative,Neutral
3,feed me or i press enter,Negative,Neutral
4,they call him gordie.,Neutral,Neutral


In [119]:
accuracy=0
count=0
predicted_labels=[]
classifier = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest")
for i in range(1250):
    res=classifier(data["text"][i])[0]
    label=res["label"]
    predicted_labels.append(label)
    if (label==to_lowercase(data["sentiment"][i])):
        count=count+1
accuracy=count/1250

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [92]:
accuracy_scores.append(accuracy)
data['Model2']=predicted_labels
data.head()

,text,sentiment,Model1,Model2
0,animals defecating fornicating and playing are...,Positive,Negative,negative
1,i regret nothing! do hear me? nothing!,Positive,Positive,positive
2,it just wanted to be a part of them,Negative,Neutral,neutral
3,feed me or i press enter,Negative,Neutral,neutral
4,they call him gordie.,Neutral,Neutral,neutral


In [96]:
accuracy=0
count=0
predicted_labels=[]
classifier = pipeline(model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
for i in range(1250):
    res=classifier(data["text"][i])[0]
    label=res["label"]
    predicted_labels.append(label)
    if (label==to_lowercase(data["sentiment"][i])):
        count=count+1
accuracy=count/1250

In [98]:
accuracy_scores.append(accuracy)
data['Model3']=predicted_labels
data.head()

,text,sentiment,Model1,Model2,Model3
0,animals defecating fornicating and playing are...,Positive,Negative,negative,neutral
1,i regret nothing! do hear me? nothing!,Positive,Positive,positive,neutral
2,it just wanted to be a part of them,Negative,Neutral,neutral,neutral
3,feed me or i press enter,Negative,Neutral,neutral,neutral
4,they call him gordie.,Neutral,Neutral,neutral,neutral


In [99]:
accuracy_scores

[0.6528, 0.6608, 0.4928]

In [100]:
import praw
reddit = praw.Reddit(
    client_id="_4w3-13ViMbAUIrtcfe8kg",
    client_secret="U9GF-bnbPYwHqLjRwqjxAenQg1ACjQ",
    user_agent="my user agent",
)
print(reddit.read_only)

True


In [102]:
headings=[]
for submission in reddit.subreddit("wallstreetbets").hot(limit=25):
    #print(submission.title)
    headings.append(submission.title)

In [104]:
for i in range(25):
    input=remove_html_tags(headings[i])
    input=remove_accented_chars(input)
    input=expand_contractions(input)
    input=remove_special_characters(input)
    #input=remove_numbers(input)
    #input=remove_punctuation(input)
    input=get_lem(input)
    #input=remove_stopwords(input)
    input=remove_extra_whitespace_tabs(input)
    input=to_lowercase(input)
    headings[i]=input

In [108]:
pos_count=0
pos_avg=0
neg_count=0
neg_avg=0
neut_count=0
neut_avg=0
predicted_labels=[]
classifier = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment")
for i in range(25):
    res=classifier(headings[i])[0]
    label=label_to_pred(res["label"])
    predicted_labels.append(label)
    if (label=="Positive"):
        pos_count=pos_count+1
        pos_avg=pos_avg+res["score"]
    if(label=="Negative"):
        neg_count=neg_count+1
        neg_avg=neg_avg+res["score"]
    if(label=="Neutral"):
        neut_count=neut_count+1
        neut_avg=neut_avg+res["score"]
if (pos_count>neg_count and pos_count>neut_count):
    print("Positive")
    print(pos_avg/pos_count)
if (neg_count>pos_count and neg_count>neg_count):
    print("Negative")
    print(neg_avg/neg_count)
if (neut_count>pos_count and neut_count>neg_count):
    print("Neutral")
    print(neut_avg/neut_count)

Neutral
0.7559383260576349
